# Agregando Datasets do SP196 em um Arquivo Consolidado

## Aviso: Comentários resumidos neste código; consulte a  [Documentação](https://github.com/DemikFR/Analise-Controle-Zoonoses-SP156/blob/main/README.md) para obter uma compreensão mais detalhada das razões por trás de cada trecho de código.

## Gerar o dataframe com os dados da Prefeitura

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from io import StringIO
import chardet

### Criar o dataframe que conterá os dados necessários para a análise

In [5]:
df = pd.DataFrame()
dfs = []

## Função para buscar os caminhos para o download de cada dataset

In [3]:
def get_soup(url, search):
    response = requests.get(url)
    
    # Verificar se a conexão foi bem sucedida
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        # Retornar as páginas onde terão as de download
        return soup.find_all('a', href=True, attrs={'title': re.compile(search)}) 
    else:
        return f'Página{link["title"]} está inacessível. Código: {response.status_code}'

## Código para inserir os dados em uma lista

In [6]:
# URL base do site da Prefeitura
url_base = 'http://dados.prefeitura.sp.gov.br'

# URL inicial onde estarão os links para os datasets
url_inicial = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'

# extensão do dataset para filtrar
file_type = '.csv'

for link in get_soup(url_inicial, r'^Dados do SP156 -.*'):
    
    # Pegar o link de download
    for page in get_soup(url_base+link['href'], r'^http://dados.prefeitura.sp.gov.br/dataset/.*'):
        link_download = page['href']
        
        # Verificar se é o arquivo a ser baixado é Excel
        if file_type in link_download:
            
            # Pegar o conteúdo do CSV
            response_csv = requests.get(page['href'])
            
            # Cada datset tem um encoding diferente, por isso foi necessário usar a biblioteca chardet para identificar 
            # automaticamente qual é.
            encoding = chardet.detect(response_csv.content)['encoding']
            
            # Colocar os dados na tabela auxiliar
            if int(link["title"][-4:]) >= 2020:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=';', encoding=encoding, low_memory=False)
            else:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=',', encoding=encoding, low_memory=False)
                
            dfs.append(df3)  # Adicionar o DataFrame intermediário à lista
            
            # Inserir os dados na planilha original
            print(f'Os dados do {link["title"]} foram inseridos com sucesso.')

Os dados do Dados do SP156 - 1º TRI 2023 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - Dezembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - Outubro e Novembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2019 foram inseridos com sucesso.
Os

## Criar o dataframe com todos os datasets coletados

In [7]:
df = pd.concat(dfs, ignore_index=True)

In [8]:
df.shape

(13862242, 33)

## Verificar todas as colunas disponibilizadas

In [9]:
df.any()

Data de abertura                                                          True
Canal                                                                     True
Tema                                                                      True
Assunto                                                                   True
Serviço                                                                   True
Logradouro                                                                True
Número                                                                    True
CEP                                                                       True
Subprefeitura                                                             True
Distrito                                                                  True
Setor                                                                     True
Quadra                                                                    True
Latitude                                            

## Manter apenas os campos necessários

In [10]:
df = df[['Data de abertura', 'Tema', 'Assunto', 'Especificação do assunto', 'Serviço', 'Bairro', 'Distrito'
         , 'Status da solicitação', 'Data do parecer']]

## Apagar registros que por terem muitos valores nulo, não servirão para a análise

In [11]:
df = df.dropna(thresh=4)

In [12]:
df[df['Data de abertura'].isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer


In [13]:
df.sample(10)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
2004868,2022-01-12 18:37:39,Trânsito e Transporte,"Ônibus - Conduta de Trabalho do Motorista, Cob...",NaN,"Ônibus – Denunciar motorista, cobrador ou fisc...",NaN,JARDIM HELENA,FINALIZADO,2022-01-20 12:50:05
6628808,2020-11-11 22:05:12,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Acolhimento para pessoas em situação de rua - ...,NaN,SE,INDEFERIDO,2020-11-12 03:48:17
1687224,2022-05-10 05:51:08,Rua e bairro,"Reparos em asfalto, pontes e viadutos",NaN,Tapa-buraco,NaN,GRAJAU,INDEFERIDO,2022-05-12 22:11:15
111609,2023-01-19 04:02:24,Cidadania e assistência social,Auxílio Brasil,NaN,Programa Auxílio Brasil - Consultar situação d...,NaN,NaN,FINALIZADA,2023-01-19 04:02:25
764784,2022-11-04 15:34:12,Finanças,Senha Web,NaN,"Senha Web - Desbloqueio online para MEI, Empre...",NaN,NaN,FINALIZADA,2022-11-04 20:09:25
9253997,2018-01-08 22:55:33.151,Rua e bairro,Capinação e roçada de áreas verdes,NaN,Praça e canteiro central - solicitar roçada,NaN,JABAQUARA,FINALIZADA,2018-08-16 13:53:35.081
8464219,2019-02-27 10:15:39.217,Lixo e limpeza,Varrição e limpeza urbana,NaN,Fazer reclamação de falta de varrição,NaN,RIO PEQUENO,FINALIZADA,2019-05-03 12:41:55.000
7420463,2020-04-01 11:36:50,Rua e bairro,Terrenos e imóveis,NaN,Denunciar falta de limpeza de terrenos e imóve...,NaN,ITAQUERA,AGUARDANDO ATENDIMENTO,NaN
2101369,2022-01-29 07:43:21,Trânsito e Transporte,"Ônibus - Conduta de Trabalho do Motorista, Cob...",NaN,Ônibus - Denunciar motorista por não atender a...,NaN,VILA MATILDE,FINALIZADO,2022-02-03 13:20:03
4112022,2021-06-01 16:52:19,Educação,Alimentação escolar,NaN,Cartão alimentação - reclamar de problemas rel...,NaN,NaN,FINALIZADA,2021-12-21 14:22:48


## Filtrar apenas os dados necessários para a análise

In [14]:
df['Tema'].drop_duplicates()

0           Cidadania e assistência social
5                                  Animais
7               Obras, imóveis e habitação
10                            Rua e bairro
34                           Meio Ambiente
37                          Lixo e limpeza
41                                Urgência
54                                   Saúde
67                     Comércio e serviços
95                   Trânsito e Transporte
151                           Causa animal
188           Ouvidoria Geral do Município
227                         Pragas urbanas
431                               Finanças
439                         Acessibilidade
918                          Bilhete Único
1272                      Segurança urbana
2334                 Documentos e licenças
2885              Cultura, esporte e lazer
4389                              Educação
8048                              Trabalho
334059              Descomplica São Miguel
596920               Canais de Atendimento
597569     

### Entender o motivo de existirem nulos no 'Tema'

In [15]:
pd.set_option('display.max_rows', None)
df[df.Tema.isnull()]['Assunto'].drop_duplicates()

11562002                             Construcao de passarelas
11562003                                               Buraco
11562004                                      Poluição Sonora
11562006    Placas de Nomes de Ruas, Praças e demais Logra...
11562008                         Iluminacao - Lampada apagada
11562009             Transporte publico / Conduta de Trabalho
11562010                              Remoção veículo/carcaça
11562012                                               Elogio
11562016                            Vistoria / Infraestrutura
11562019                                                Obras
11562020                                           Edificacao
11562021                                        Animais / Cão
11562026                                         Pavimentacao
11562031                      Árvore(Calçada/Passeio Público)
11562034                          Transporte publico / Outros
11562036                                       Guias/sarjetas
11562038

### O campo 'Tema' só é usado em datasets mais recentes, por isso deve-se utilizar o campo 'Assunto' para filtrar

In [16]:
df1 = df[df['Assunto'].str.contains('Animais', regex=False) | df['Tema'].str.contains(r'^Anim.*|^Pragas.*', regex=True)].reset_index(drop=True)

In [17]:
df1.shape

(704360, 9)

In [18]:
df1.sample(20)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
664147,2012-12-03 12:18:56.893,NaN,Animais / Cão,Atropelado ou Acidentado vivo e sem proprietário,NaN,JARDIM RODOLFO PIRANI,77.0,REALIZADA,2017-05-09 13:31:35.960000
13919,2023-02-27 08:23:23,Pragas urbanas,Animais que transmitem doenças ou risco à saúde,NaN,Escorpião - Solicitar vistoria de local com es...,NaN,BRASILANDIA,AGUARDANDO ATENDIMENTO,2023-02-28 04:46:45
459263,2016-02-04 10:02:55.083,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,SANTANA,FINALIZADA,2016-04-01 18:44:56.343
23534,2022-10-22 19:57:50,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2022-11-06 09:49:05
371506,2019-03-27 19:39:18.965,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Pernilongo/Mosquito - Solicitar vistoria em lo...,NaN,ITAQUERA,FINALIZADA,2019-04-22 15:25:19.031
481174,2015-08-07 16:12:54.010,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,CURSINO,FINALIZADA,2015-10-15 03:00:00.000
496705,2015-12-03 12:47:42.390,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,RIO PEQUENO,FINALIZADA,2016-10-26 17:58:05.250
466066,2016-03-21 12:54:48.260,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,SANTO AMARO,FINALIZADA,2016-09-06 03:00:00.000
527473,2015-06-12 17:12:20.910,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,JARDIM PAULISTA,FINALIZADA,2015-06-16 19:23:50.937
130736,2021-10-11 03:57:10,Animais,Registro de animais - RGA,NaN,Avisar sobre animal encontrado com Registro G...,NaN,ERMELINO MATARAZZO,FINALIZADA,2021-11-12 14:15:45


## Padronizar o campo 'Serviço' com o 'Especificação do assunto'

#### Os dois campos possuem o mesmo objetivo

In [19]:
df1.loc[df1['Serviço'].isnull(), 'Serviço'] = df1.loc[df1['Serviço'].isnull(), 'Especificação do assunto']

In [20]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
169917,2021-08-03 20:08:14,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2021-08-04 14:20:58
302639,2020-08-12 17:31:47,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2020-09-13 13:13:49
300901,2020-08-06 15:29:49,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2020-09-01 13:03:32
307050,2020-08-29 00:03:10,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2020-09-29 13:31:45
253253,2021-03-09 23:13:54,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2021-03-31 14:17:53
31008,2022-11-23 13:12:46,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,NaN,FINALIZADA,2022-12-06 14:16:05
298741,2020-07-29 13:13:04,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2020-08-06 17:54:20
134727,2021-10-25 01:35:08,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2021-11-05 17:07:36
493681,2015-11-16 14:15:54.783,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,SAUDE,FINALIZADA,2016-04-27 03:00:00.000
150694,2021-12-03 12:54:28,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2021-12-03 14:42:56


In [21]:
df1[df1['Serviço'].isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer


#### Deletar o campo que não terá mais utilidade

In [22]:
df1 = df1.drop('Especificação do assunto', axis=1)

## Tratamento dos campos de localização

#### Os campos 'Bairro' e 'Distrito' são ambíguos, por isso que os valores que estão nulos em 'Distrito' receberão o valor de 'Bairro', caso exista. Além disso, o campo 'Distrito' existe alguns que são número, indicando um distrito que está na metadado disponibilizado neste repositório.

### Preencher valores de 'Distrito' com o de 'Bairro'

In [23]:
df1.loc[df1['Distrito'].isnull() & df1['Bairro'].notnull(), 'Distrito'] = df1.loc[df1['Distrito'].isnull() & df1['Bairro'].notnull(), 'Bairro']

In [24]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
410477,2018-04-02 21:14:27.028,Animais,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado,NaN,SAO MIGUEL,FINALIZADA,2018-04-23 17:53:23.739
306271,2020-08-26 15:47:18,Animais,Animais que transmitem doenças ou risco à saúde,Pombos - Solicitar vistoria em local infestado,NaN,CIDADE LIDER,FINALIZADA,2020-08-31 12:48:30
9275,2023-01-29 08:22:19,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2023-01-30 11:55:06
549906,2014-12-02,NaN,Animais / Ratos,Reclamação de ratos,VILA JAGUARI,VILA JAGUARI,INDEFERIDA,2015-09-15 03:00:00.000
484757,2015-09-11 11:06:23.497,Animais,Animal em via pública,Solicitar remoção de animal morto em via pública,NaN,CAMPO LIMPO,FINALIZADA,2015-09-15 11:47:54.663
461290,2016-02-17 13:08:03.613,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,SACOMA,FINALIZADA,2016-04-05 03:00:00.000
531742,2014-07-26,NaN,Animais / Ratos,Reclamação de ratos,JARDIM DO CAMPO,JARDIM DO CAMPO,REALIZADA,2014-07-28 00:00:00.000
318558,2020-09-30 20:21:22,Animais,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...,NaN,PINHEIROS,FINALIZADA,2020-10-10 18:12:57
621399,2013-02-26,NaN,Animais / Ratos,Reclamação de ratos,JARDIM UMARIZAL,JARDIM UMARIZAL,REALIZADA,2013-03-05 00:00:00.000
569698,2014-03-15,NaN,"Animais / Abelhas, Vespas, Marimbondos",Colméia/Vespeiro instalado,JARDIM MARILU,JARDIM MARILU,REALIZADA,2014-04-12 00:00:00.000


### Converter os valores que estão em código para o seu nome original

In [25]:
distrito = pd.read_csv('codigos_de_distritos.csv', sep=';')

In [26]:
distrito['Nr Distrito'] = distrito['Nr Distrito'].astype(float)

In [27]:
distrito

,Nr Distrito,Nome Distrito
0,1.0,Agua Rasa
1,2.0,Alto de Pinheiros
2,3.0,Anhanguera
3,4.0,Aricanduva
4,5.0,Artur Alvim
5,6.0,Barra Funda
6,7.0,Bela Vista
7,8.0,Belem
8,9.0,Bom Retiro
9,10.0,Bras


In [28]:
df1['Distrito'] = df1['Distrito'].replace(distrito.set_index('Nr Distrito')['Nome Distrito'])

In [29]:
df1 = df1.drop('Bairro', axis=1)

In [30]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Serviço,Distrito,Status da solicitação,Data do parecer
600891,2013-11-06,NaN,Animais / Ratos,Reclamação de ratos,PARQUE REBOUCAS,REALIZADA,2013-11-28 00:00:00.000
512541,2015-03-03 16:34:01.730,Animais,Animal agressor e/ou invasor,Cães e gatos invasores de instituições pública...,GRAJAU,CANCELADA,2015-03-17 03:00:00.000
219806,2021-06-16 11:03:16,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-06-16 08:15:51
287289,2020-11-20 04:45:25,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2020-12-06 16:36:29
384652,2018-08-01 19:39:31.568,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e Vespas - Solicitar vistoria em ninhos,JARAGUA,FINALIZADA,2018-08-15 13:43:03.040
465729,2016-03-17 21:05:53.150,Animais,Animal agressor e/ou invasor,Avisar sobre animal agressor em vias públicas,SANTANA,FINALIZADA,2016-04-12 03:00:00.000
310625,2020-09-11 10:57:35,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2020-10-25 12:20:16
493862,2015-11-17 12:13:24.213,Animais,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado,VILA MEDEIROS,FINALIZADA,2016-04-20 03:00:00.000
25812,2022-10-31 14:48:41,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2022-11-03 08:26:45
533267,2014-08-11,NaN,Animais / Gato,Gato agressor,JARDIM CIDADE PIRITUBA,CANCELADA,2014-08-25 00:00:00.000


In [31]:
df1['Distrito'].isnull().sum()

232507

In [32]:
df1['Distrito'].notnull().sum()

471853

## Tratamento das descrições (Assunto, Serviço)

In [33]:
df2 = df1.rename({'Assunto': 'Tipo de Serviço'}, axis=1)

In [34]:
df2.sample()

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
198020,2021-04-21 22:08:17,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-05-03 11:00:37


In [35]:
df2['Tipo de Serviço'].drop_duplicates()

0                                Registro de animais - RGA
1                              Exames, vacinas e castração
3                                        Adoção de animais
4                            Criação inadequada de animais
6                                       Animais silvestres
9                                    Animal em via pública
14         Animais que transmitem doenças ou risco à saúde
23                            Animal agressor e/ou invasor
66        Dengue/chikungunya/zika (mosquito aedes aegypti)
26265                                  Comércio de animais
128038                    Animais que podem causar doenças
361615                                      Animais/Aranha
386661                                  Animais/Carrapatos
392961                                      Animais/Pulgas
397007                                      Animais/barata
529214                                       Animais / Cão
529218                                      Animais / Ga

In [36]:
df2[['Tipo de Serviço', 'Serviço']].drop_duplicates()

,Tipo de Serviço,Serviço
0,Registro de animais - RGA,Avisar sobre animal encontrado com Registro G...
1,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente
3,Adoção de animais,Adotar cães e gatos
4,Criação inadequada de animais,Denunciar condições inadequadas de criação
6,Animais silvestres,Animais silvestres feridos ? Solicitar atendim...
9,Animal em via pública,Solicitar remoção de animal morto em via pública
12,Animal em via pública,Avaliação de animal em sofrimento sem propriet...
14,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...
15,Animais que transmitem doenças ou risco à saúde,Morcegos - Reclamar sobre local com morcegos
19,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado


### Padronizar o Tipo de Serviço

In [37]:
df2.loc[df2['Tipo de Serviço'].str.contains('Animais[ |/]') & df2['Tipo de Serviço']
        .str.match('^(?!.*(Cão|Gato|Cavalo|silvestres|RGA|agressor|via))'), 'Tipo de Serviço'
       ] = 'Animais que transmitem doenças ou risco à saúde'

In [38]:
df2[['Tipo de Serviço', 'Serviço']].drop_duplicates()

,Tipo de Serviço,Serviço
0,Registro de animais - RGA,Avisar sobre animal encontrado com Registro G...
1,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente
3,Adoção de animais,Adotar cães e gatos
4,Criação inadequada de animais,Denunciar condições inadequadas de criação
6,Animais silvestres,Animais silvestres feridos ? Solicitar atendim...
9,Animal em via pública,Solicitar remoção de animal morto em via pública
12,Animal em via pública,Avaliação de animal em sofrimento sem propriet...
14,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...
15,Animais que transmitem doenças ou risco à saúde,Morcegos - Reclamar sobre local com morcegos
19,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado


In [39]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace('Animais / ', '')

In [40]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace(' / Animais', '')

In [41]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace('Dengue/chikungunya/zika (mosquito aedes aegypti)', 'Vistoria')

In [42]:
df2['Tipo de Serviço'].drop_duplicates()

0                               Registro de animais - RGA
1                             Exames, vacinas e castração
3                                       Adoção de animais
4                           Criação inadequada de animais
6                                      Animais silvestres
9                                   Animal em via pública
14        Animais que transmitem doenças ou risco à saúde
23                           Animal agressor e/ou invasor
66                                               Vistoria
26265                                 Comércio de animais
529214                                                Cão
529218                                               Gato
529226                                            Animais
529765                                             Cavalo
Name: Tipo de Serviço, dtype: object

### Apagar os registros de teste

In [43]:
df2 = df2.drop(df2.loc[df1['Serviço'].str.contains('TESTE', regex=True)].index)

### Padronizar o Serviço

#### Tratar o Tipo de Serviço 'Animais que transmitem doenças ou risco à saúde'

In [44]:
df2.loc[df2['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

21          Abelhas e Vespas - Solicitar vistoria em ninhos
529220                           Colméia/Vespeiro instalado
69        Escorpião - Solicitar vistoria de local com es...
15             Morcegos - Reclamar sobre local com morcegos
529296                   Ocorrências / Condições de criação
529237                              Ocorrências com morcego
585177                               Ocorrências rotineiras
14        Pernilongo/Mosquito - Solicitar vistoria em lo...
227          Pombos - Solicitar vistoria em local infestado
19            Ratos - Solicitar vistoria em local infestado
361615                                Reclamação de Aranhas
529472                                Reclamação de Baratas
529263                             Reclamação de Carrapatos
540659                             Reclamação de Escorpiões
529244                  Reclamação de Escorpiões ou Aranhas
529414                               Reclamação de Formigas
529725                                 R

In [45]:
df3 = df2.copy()

#### Juntar Serviços iguais com nomes diferentes

In [110]:
df3.loc[(df3['Serviço'].str.contains('Pernilongo/Mosquito', regex=True)) | (df3['Serviço'] == 'Reclamação de Pernilongo'), 'Serviço'] = 'Reclamação de Pernilongos e Mosquitos'

In [47]:
df3.loc[(df3['Serviço'] == 'Abelhas e Vespas') | (df3['Serviço'] == 'Colméia/Vespeiro instalado'), 'Serviço'] = 'Remoção de Abelhas, Vespas ou Marimbondos'

In [48]:
df3.loc[df3['Serviço'].str.contains('Escorpião', regex=True), 'Serviço'] = 'Escorpiões'

In [88]:
df3.loc[df3['Serviço'] == 'Reclamação de Escorpião', 'Serviço'] = 'Reclamação de Escorpiões'

In [50]:
df3.loc[df3['Serviço'] == 'Ocorrências com morcego', 'Serviço'] = 'Reclamação de Morcegos'

#### Juntar diversos Serviços que estão duplicados, seguindo um padrão igual

In [51]:
df3.loc[df3['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'] = df3.loc[df3[
    'Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].str.replace(r' -.*', '', regex=True)

In [52]:
df3.loc[df3['Serviço'].str.split().str.len() == 1, 'Serviço'] = 'Reclamação de ' + df3['Serviço']

In [53]:
df3.loc[df3['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

21                                 Abelhas e Vespas
529296           Ocorrências / Condições de criação
585177                       Ocorrências rotineiras
361615                        Reclamação de Aranhas
529472                        Reclamação de Baratas
529263                     Reclamação de Carrapatos
69                          Reclamação de Escorpião
540659                     Reclamação de Escorpiões
529244          Reclamação de Escorpiões ou Aranhas
529414                       Reclamação de Formigas
15                           Reclamação de Morcegos
529725                         Reclamação de Moscas
14                Reclamação de Pernilongo/Mosquito
529229        Reclamação de Pernilongos e Mosquitos
227                            Reclamação de Pombos
529482                         Reclamação de Pulgas
19                              Reclamação de Ratos
397007                        Reclamação de baratas
386661                     Reclamação de carrapatos
392961      

#### Tratar os Tipos de Serviços que NÃO são 'Animais que transmitem doenças ou risco à saúde'

#### Retirar caracteres não úteis

In [54]:
df3['Serviço'] = df3['Serviço'].str.replace('?', '-')
df3['Serviço'] = df3['Serviço'].str.replace('–', '-')
df3['Serviço'] = df3['Serviço'].str.replace('Invadiu o local ', 'Invadiu o local')

In [55]:
pd.set_option('display.max_colwidth', None)
df3.loc[df3['Tipo de Serviço'] != 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

529406                                                          Acidentado sem proprietário
3                                                                       Adotar cães e gatos
1460                                        Animais silvestres - Agendar entrega voluntária
6                            Animais silvestres feridos - Solicitar atendimento veterinário
109                                        Animal de grande porte solto - Solicitar remoção
529221                                     Atropelado ou Acidentado vivo e sem proprietário
529238                                                          Atropelado sem proprietário
12        Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)
0                        Avisar  sobre animal encontrado com Registro Geral do Animal (RGA)
23                                            Avisar sobre animal agressor em vias públicas
34365                  Cadastro Municipal de Comércio de Animais (CMCA) - Cancel

#### Padronizar os 'Serviços' que contém 'Acidentado' ou 'Atropelado'

In [56]:
df3.loc[df3['Serviço'].str.contains('Atropelado') | df3['Serviço'].str.contains('Acidentado'), 'Serviço'
       ] = 'Atropelado ou Acidentado vivo e sem proprietário'

#### Padronizar 'Condições de criação' e 'Denunciar condições inadequadas de criação' com 'Condições de criação / maus tratos'

In [57]:
df3.loc[df3['Serviço'].str.match('.*ondições'), 'Serviço'
       ] = 'Condições de criação / maus tratos'

#### Padronizar os que tem 'Remoção de animal morto em via pública' (3)

In [58]:
df3.loc[df3['Serviço'].str.match('.*morto em via.*'), 'Serviço'
       ] = 'Remoção de animal morto em via pública'

#### Usar o 'Tipo de Serviço' para preencher 'Serviços' não tão bem descritos, como 'Solto em via pública'

In [59]:
condicoes = (df3['Serviço'] == 'Em parques') | (df3['Serviço'] == 'Invadiu o local') | (df3['Serviço'] == 'Solto em via pública')
df3.loc[condicoes, 'Serviço'] = df3.loc[condicoes, 'Tipo de Serviço'].str.cat(df3.loc[condicoes, 'Serviço'], sep=' ')

In [60]:
condicoes = (df3['Serviço'] == 'Diversas ocorrências') | (df3['Serviço'] == 'Ocorrências rotineiras')
df3.loc[condicoes, 'Serviço'] = df3.loc[condicoes, 'Tipo de Serviço'].str.cat(df3.loc[condicoes, 'Serviço'], sep=' - ')

In [61]:
pd.set_option('display.max_colwidth', None)
df3.loc[df3['Tipo de Serviço'] != 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

3                                                                       Adotar cães e gatos
1460                                        Animais silvestres - Agendar entrega voluntária
6                            Animais silvestres feridos - Solicitar atendimento veterinário
109                                        Animal de grande porte solto - Solicitar remoção
529221                                     Atropelado ou Acidentado vivo e sem proprietário
12        Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)
0                        Avisar  sobre animal encontrado com Registro Geral do Animal (RGA)
23                                            Avisar sobre animal agressor em vias públicas
34365                  Cadastro Municipal de Comércio de Animais (CMCA) - Cancelar cadastro
26265                    Cadastro Municipal de Comércio de Animais (CMCA) - Fazer inscrição
501                                        Castrar cães e gatos de protetores in

In [62]:
df3[['Tipo de Serviço', 'Serviço']].drop_duplicates().sort_values(by=['Tipo de Serviço', 'Serviço'])

,Tipo de Serviço,Serviço
3,Adoção de animais,Adotar cães e gatos
529226,Animais,Remoção de animal morto em via pública
21,Animais que transmitem doenças ou risco à saúde,Abelhas e Vespas
585177,Animais que transmitem doenças ou risco à saúde,Animais que transmitem doenças ou risco à saúde - Ocorrências rotineiras
529296,Animais que transmitem doenças ou risco à saúde,Condições de criação / maus tratos
361615,Animais que transmitem doenças ou risco à saúde,Reclamação de Aranhas
529472,Animais que transmitem doenças ou risco à saúde,Reclamação de Baratas
529263,Animais que transmitem doenças ou risco à saúde,Reclamação de Carrapatos
69,Animais que transmitem doenças ou risco à saúde,Reclamação de Escorpião
540659,Animais que transmitem doenças ou risco à saúde,Reclamação de Escorpiões


In [63]:
df3.sample(10)

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
69232,2022-04-07 08:56:32,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2022-04-11 15:48:54
607057,2013-12-18,NaN,Animais que transmitem doenças ou risco à saúde,"Remoção de Abelhas, Vespas ou Marimbondos",JARDIM SAO PAULO,REALIZADA,2013-12-23 00:00:00.000
501809,2015-01-05 16:58:21.810,Animais,Criação inadequada de animais,Condições de criação / maus tratos,PERDIZES,FINALIZADA,2015-01-06 02:00:00.000
284341,2020-11-12 20:15:29,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2020-11-30 12:12:28
294538,2020-07-15 13:02:52,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2020-08-06 16:06:48
311966,2020-09-14 18:03:23,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2020-11-04 15:52:37
499877,2015-12-21 13:06:40.587,Animais,Animal agressor e/ou invasor,Avisar sobre animal agressor em vias públicas,NaN,FINALIZADA,2017-06-14 17:45:00.140
225728,2021-07-01 00:02:16,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-07-01 11:06:24
482457,2015-08-19 19:35:52.093,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e Vespas,SAO MIGUEL,FINALIZADA,2015-09-18 16:30:59.397
251584,2021-03-04 19:10:19,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-03-05 14:16:06


### Realizar um capitalize em cada "Serviço" caso gouver necessidade

In [105]:
def capitalize(servico):
    # Descobrir o sinal que a frase usa
    sinal = re.findall(r" - | / ", servico)
    
    if sinal:
        # Dividindo a frase em duas partes: antes e depois do traço ou da barra
        partes = re.split(sinal[0], servico)
        
        # Realizando o capitalize na primeira parte (antes do traço)
        parte1 = partes[0].capitalize()

        # Realizando o capitalize na segunda parte (depois do traço)
        parte2 = partes[1].capitalize()
        
        return "".join([parte1, sinal[0], parte2])
    else:
        return servico.capitalize()

In [121]:
df3['Serviço'] = df3['Serviço'].apply(lambda x: capitalize(x) if not re.search(r'RGA|CMCA', x) else x)

In [122]:
df3[['Tipo de Serviço', 'Serviço']].drop_duplicates().sort_values(by=['Tipo de Serviço', 'Serviço'])

,Tipo de Serviço,Serviço
3,Adoção de animais,Adotar cães e gatos
529226,Animais,Remoção de animal morto em via pública
21,Animais que transmitem doenças ou risco à saúde,Abelhas e vespas
585177,Animais que transmitem doenças ou risco à saúde,Animais que transmitem doenças ou risco à saúde - Ocorrências rotineiras
529296,Animais que transmitem doenças ou risco à saúde,Condições de criação / Maus tratos
361615,Animais que transmitem doenças ou risco à saúde,Reclamação de aranhas
397007,Animais que transmitem doenças ou risco à saúde,Reclamação de baratas
386661,Animais que transmitem doenças ou risco à saúde,Reclamação de carrapatos
69,Animais que transmitem doenças ou risco à saúde,Reclamação de escorpiões
529244,Animais que transmitem doenças ou risco à saúde,Reclamação de escorpiões ou aranhas


### Transformar todas as datas e horas para apenas data

In [123]:
df4 = df3.copy()

#### 'Data de abertura'

In [124]:
df4['Data de abertura'] = df4['Data de abertura'].str.slice(0,10)

In [125]:
df4['Data de abertura'] = pd.to_datetime(df4['Data de abertura']).dt.date

In [127]:
df4['Data de abertura'].sample(10)

5427      2023-01-17
669043    2012-12-29
372339    2019-04-04
529747    2014-07-06
272281    2020-10-07
232770    2021-01-17
228501    2021-01-07
265662    2020-12-14
40701     2022-07-03
530358    2014-07-14
Name: Data de abertura, dtype: object

In [128]:
df4.loc[df4['Data do parecer'].notnull(), 'Data do parecer'] = df4.loc[
    df4['Data do parecer'].notnull(), 'Data do parecer'].str.slice(0,10)

In [129]:
df4['Data do parecer'].sample(10)

336006    2020-06-19
373961    2019-07-01
17616     2023-04-03
322114    2020-03-05
579186    2014-05-08
98989     2022-02-01
462567    2016-03-28
153966    2021-12-17
204073    2021-05-14
564913    2014-03-07
Name: Data do parecer, dtype: object

In [130]:
df4.sample(10)

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
89552,2022-06-12,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2022-06-13
214102,2021-06-02,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-06-02
2545,2023-01-09,Animais,Animais que transmitem doenças ou risco à saúde,Reclamação de ratos,CIDADE DUTRA,FINALIZADA,2023-01-10
18139,2022-10-03,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2022-10-10
482939,2015-08-25,Animais,Animais que transmitem doenças ou risco à saúde,Reclamação de pernilongos e mosquitos,JARDIM HELENA,FINALIZADA,2015-08-28
93485,2022-06-26,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2022-07-04
641087,2013-06-25,NaN,Cão,Atropelado ou acidentado vivo e sem proprietário,PARQUE NOVO MUNDO,REALIZADA,2017-05-17
527007,2015-06-09,Animais,Animais que transmitem doenças ou risco à saúde,Reclamação de pombos,AGUA RASA,FINALIZADA,2015-11-09
76191,2022-04-29,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e vespas,JOSE BONIFACIO,FINALIZADA,2022-05-20
315935,2020-09-25,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2020-11-03


In [131]:
df4.isnull().any()

Data de abertura         False
Tema                      True
Tipo de Serviço          False
Serviço                  False
Distrito                  True
Status da solicitação    False
Data do parecer           True
dtype: bool

## Gerar o arquivo com os dados necessários

In [132]:
df4.to_parquet('sp156_all_time.gzip',
              compression='gzip',
              index=False)